In [42]:
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np
import pandas as pd

$$
PV_{\text{Default Leg}} = LGD \times \sum_{i=1}^{n3Y} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right]
$$

$$
PV_{\text{Coupon Leg}} = s \times \sum_{i=1}^{n3Y} \alpha_i \cdot P(t, t_i) \cdot e^{-\lambda_{3Y} \cdot (t_i - t)}
$$


$$
UF^{3Y} = PV_{\text{Default Leg}} - PV_{\text{Coupon Leg}}
$$

In [43]:
# Definizione dei parametri del problema
r = 0.03  # Tasso di interesse zero-coupon annuale
recovery_rate = 0.4  # Tasso di recupero
LGD = 1 - recovery_rate  # Loss Given Default (1 - tasso di recupero)
spread = 0.01  # Spread del CDS, cedola annuale
n3Y = 3  # Numero di anni fino alla scadenza del CDS a 3 anni
n5Y = 5  # Numero di anni fino alla scadenza del CDS a 5 anni
upfront_3y = 0.068  # Premio upfront osservato per il CDS a 3 anni
upfront_5y = 0.096  # Premio upfront osservato per il CDS a 5 anni


Definiamo una funzione per calcolare il prezzo di un zero-coupon bond. Questo è usato per scontare i flussi di cassa futuri al valore presente.

In [44]:
# Funzione che calcola il prezzo di uno zero-coupon bond che matura al tempo t
def zero_coupon_bond_price(r, t):
    return np.exp(-r * t)

In [45]:
# Dati aggiuntivi per il problema
upfront_observed_3y = upfront_3y  # Premio upfront osservato per il CDS a 3 anni

# Funzione che calcola il valore attuale del premio upfront per un CDS a 3 anni secondo la formula fornita
def pv_upfront_3y_new(lambda_3Y):
    # Calcolo del valore attuale dei flussi di cassa per la perdita data il default (LGD)
    pv_loss = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    
    # Calcolo del valore attuale dei flussi di cassa dei coupon pagati
    pv_coupon_payments = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    
    # Valore attuale del premio upfront calcolato
    pv_upfront_calculated = LGD * pv_loss - pv_coupon_payments
    
    # La differenza tra il valore attuale calcolato e il premio upfront osservato
    return pv_upfront_calculated - upfront_observed_3y

lambda_3Y_initial_guess = 0.1
# Risolviamo per lambda_3Y che zera la funzione pv_upfront_3y_new
lambda_3Y_solution = fsolve(pv_upfront_3y_new, lambda_3Y_initial_guess)

lambda_3Y_solution[0]*100



5.993478603980348

$$
% Definizione del valore attuale del premio upfront per il CDS a 5 anni
PV_{\text{upfront, 5y}}(\lambda_{5Y}) = LGD \left( \sum_{i=1}^{3} P(t, t_i) \left[ e^{-\lambda_{3Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{3Y} \cdot (t_i - t)} \right] + \sum_{i=4}^{5} P(t, t_i) \left[ e^{-\lambda_{5Y} \cdot (t_{i-1} - t)} - e^{-\lambda_{5Y} \cdot (t_i - t)} \right] \right) - s \left( \sum_{i=1}^{3} \alpha_i P(t, t_i) e^{-\lambda_{3Y} \cdot (t_i - t)} + \sum_{i=4}^{5} \alpha_i P(t, t_i) e^{-\lambda_{5Y} \cdot (t_i - t)} \right)
\\
\\
% Condizione che il valore attuale calcolato sia uguale al premio upfront osservato sul mercato
UF^{5Y} = PV_{\text{upfront, 5y}}(\lambda_{5Y})
$$

In [46]:
# Numero di periodi di pagamento fino alla scadenza del CDS a 5 anni
n5Y = 5

# Premio upfront osservato per il CDS a 5 anni
upfront_observed_5y = upfront_5y  # Già definito precedentemente come 0.096

# Calcoliamo il valore attuale dei flussi di cassa fino a 3 anni usando lambda_3Y trovato
lambda_3Y = lambda_3Y_solution[0]  # Utilizziamo il lambda_3Y trovato precedentemente

# Funzione per il calcolo del valore attuale del premio upfront per il CDS a 5 anni
def pv_upfront_5y_new(lambda_5Y):
    # Flussi di cassa fino a 3 anni, usando lambda_3Y
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_coupon_payments_3y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    
    # Aggiungiamo i flussi di cassa dagli anni 3 a 5, usando lambda_5Y
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_coupon_payments_5y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    
    # Valore attuale totale del premio upfront calcolato
    pv_upfront_calculated = (LGD * (pv_loss_3y + pv_loss_5y)) - (pv_coupon_payments_3y + pv_coupon_payments_5y)
    
    # La differenza tra il valore attuale calcolato e il premio upfront osservato per il CDS a 5 anni
    return pv_upfront_calculated - upfront_observed_5y

# Valore iniziale stimato per lambda_5Y per iniziare la ricerca della soluzione
lambda_5Y_initial_guess = 0.1

# Utilizzo di fsolve per trovare il valore di lambda_5Y
lambda_5Y_solution = fsolve(pv_upfront_5y_new, lambda_5Y_initial_guess)

lambda_5Y_solution[0] *100 


4.868322413146313

Questa formula fornisce la rappresentazione matematica di come calcoliamo la probabilità di sopravvivenza per il CDS, sia per i primi tre anni utilizzando 
$\lambda_{3Y}$ sia per gli anni successivi fino a cinque anni utilizzando $\lambda_{5Y}$

$$
% Calcolo della probabilità di sopravvivenza fino a un certo tempo T
\text{Surv}(T) = 
\begin{cases} 
e^{-\lambda_{3Y} \cdot T}, & \text{se } T \leq 3 \\
e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (T - 3)}, & \text{se } T > 3 
\end{cases}

$$

$$
\text{Probabilità di sopravvivenza a 1 anno} = e^{-\lambda_{3Y} \cdot 1}\\
\text{Probabilità di sopravvivenza a 2 anni} = e^{-\lambda_{3Y} \cdot 2}\\
\text{Probabilità di sopravvivenza a 3 anni} = e^{-\lambda_{3Y} \cdot 3}\\
\text{Probabilità di sopravvivenza a 4 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (4 - 3)}\\
\text{Probabilità di sopravvivenza a 5 anni} = e^{-\lambda_{3Y} \cdot 3} \cdot e^{-\lambda_{5Y} \cdot (5 - 3)}
$$



Utilizziamo la proprietà dei logaritmi e degli esponenziali per combinare le probabilità di sopravvivenza in intervalli di tempo consecutivi, assumendo che l'intensità di default sia costante in ciascun intervallo. Questo ci permette di calcolare la probabilità complessiva che la controparte sopravviva senza default fino al tempo $T$, dato che ha sopravvissuto fino a 3 anni, e poi calcolare la probabilità che continui a sopravvivere fino a $T$ utilizzando $\lambda_{5Y}$

In [47]:
# Funzione per calcolare la probabilità di sopravvivenza fino a un certo tempo T
def calculate_survival_probability(lambda_1, lambda_2, T, n3Y):
    # Se T è entro i primi 3 anni, usiamo solo lambda_1
    if T <= n3Y:
        return np.exp(-lambda_1 * T) # Equivale a e^(-∫ from 0 to T of lambda_1 dt)
    # Altrimenti, usiamo lambda_1 per i primi 3 anni e lambda_2 per i restanti anni fino a T
    else:
        return np.exp(-lambda_1 * n3Y) * np.exp(-lambda_2 * (T - n3Y)) # Equivale a e^(-∫ from 0 to 3 of lambda_1 dt) * e^(-∫ from 3 to T of lambda_2 dt)

# Calcoliamo le probabilità di sopravvivenza per ciascun anno
prob_survival_1y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 1, n3Y)
prob_survival_2y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 2, n3Y)
prob_survival_3y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 3, n3Y)
prob_survival_4y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 4, n3Y)
prob_survival_5y = calculate_survival_probability(lambda_3Y, lambda_5Y_solution[0], 5, n3Y)

(prob_survival_1y, prob_survival_2y, prob_survival_3y, prob_survival_4y, prob_survival_5y)


data = [['1y', prob_survival_1y, prob_survival_1y*100],
        ['2y', prob_survival_2y, prob_survival_2y*100],
        ['3y', prob_survival_3y, prob_survival_3y*100], 
        ['4y', prob_survival_4y, prob_survival_4y*100], 
        ['5y', prob_survival_5y, prob_survival_5y*100]] 
df = pd.DataFrame(data, columns=['Year', 'Prob. Survival', '% Prob. Survival'])
df = df.style.set_caption('Probability of survival')
df


,Year,Prob. Survival,% Prob. Survival
0,1y,0.941826,94.182595
1,2y,0.887036,88.703612
2,3y,0.835434,83.543364
3,4y,0.795736,79.573618
4,5y,0.757925,75.792503


---

In [48]:
# r = 0.03  # Tasso di interesse zero-coupon annuale
# recovery_rate = 0.4  # Tasso di recupero
# LGD = 1 - recovery_rate  # Loss Given Default
# spread = 0.01  # Spread del CDS, che è pari al tasso di cedola annuale
# n3Y = 3  # Numero di periodi di pagamento fino alla scadenza del CDS a 3 anni
# n5Y = 5  # Numero di periodi di pagamento fino alla scadenza del CDS a 5 anni
# upfront_observed_3y = 0.068  # Premio upfront osservato per il CDS a 3 anni
# upfront_observed_5y = 0.096  # Premio upfront osservato per il CDS a 5 anni

# # Definisco la funzione per il prezzo di uno zero-coupon bond
# def zero_coupon_bond_price(r, t):
#     return np.exp(-r * t)

# Definisco la funzione per calcolare il valore attuale del premio upfront per il CDS a 3 anni
def pv_upfront_3y(lambda_3Y):
    pv_loss = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_cedole = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    return LGD * pv_loss - pv_cedole - upfront_observed_3y

# Calcolo lambda_3Y usando fsolve
lambda_3Y_initial_guess = 0.1
lambda_3Y_solution = fsolve(pv_upfront_3y, lambda_3Y_initial_guess)
lambda_3Y = lambda_3Y_solution[0]

# Definisco la funzione per calcolare il valore attuale del premio upfront per il CDS a 5 anni
def pv_upfront_5y(lambda_5Y):
    pv_loss_3y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_3Y * (i - 1)) - np.exp(-lambda_3Y * i)) for i in range(1, n3Y + 1)])
    pv_cedole_3y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_3Y * i) for i in range(1, n3Y + 1)])
    pv_loss_5y = sum([zero_coupon_bond_price(r, i) * (np.exp(-lambda_5Y * (i - 1)) - np.exp(-lambda_5Y * i)) for i in range(n3Y + 1, n5Y + 1)])
    pv_cedole_5y = sum([spread * zero_coupon_bond_price(r, i) * np.exp(-lambda_5Y * i) for i in range(n3Y + 1, n5Y + 1)])
    return LGD * (pv_loss_3y + pv_loss_5y) - (pv_cedole_3y + pv_cedole_5y) - upfront_observed_5y

# Calcolo lambda_5Y usando fsolve
lambda_5Y_initial_guess = 0.1
lambda_5Y_solution = fsolve(pv_upfront_5y, lambda_5Y_initial_guess)
lambda_5Y = lambda_5Y_solution[0]


# lambda_3Y, lambda_5Y 

# Calcolo e stampo i valori del premio upfront calcolati
upfront_3y_calculated = pv_upfront_3y(lambda_3Y) + upfront_observed_3y
upfront_5y_calculated = pv_upfront_5y(lambda_5Y) + upfront_observed_5y

upfront_3y_calculated, upfront_5y_calculated


(0.06800000000000013, 0.09599999999999999)

In [49]:
lambda_3Y*100, lambda_5Y*100

(5.993478603980348, 4.868322413146313)

In [50]:
upfront_observed_3y, upfront_3y_calculated, upfront_3y_calculated-upfront_observed_3y

(0.068, 0.06800000000000013, 1.249000902703301e-16)

In [51]:
upfront_observed_5y, upfront_5y_calculated, upfront_5y_calculated-upfront_observed_5y

(0.096, 0.09599999999999999, -1.3877787807814457e-17)

---

---

Il valore di un'opzione call nel modello di Bachelier è dato da:
$C = (S_0 - K) \cdot N(d_1) + \sigma \cdot \sqrt{T} \cdot n(d_1)$

dove:
- $S_0$ è il prezzo corrente del sottostante.
- $K$ è il prezzo di esercizio.
- $N(d)$ è la funzione di distribuzione cumulativa normale standard.
- $n(d)$ è la funzione di densità di probabilità normale standard.
- $d_1 = \frac{S_0 - K}{\sigma \sqrt{T}}$.
- $\sigma$ è la volatilità del sottostante.
- $T$ è il tempo alla scadenza.

La formula per il CVA sarà poi:
$CVA = LGD \cdot (C \cdot e^{-rT})$

In [52]:
# Definizione dei parametri di base per il calcolo del CVA
r = 0.03  # Tasso di interesse zero-coupon annuale da Parte I
lgd = 0.60  # Loss Given Default
X0 = 2000  # Valore iniziale del sottostante
sigma_X = 300  # Volatilità del sottostante in termini annuali
K = 1900  # Prezzo di esercizio del forward
T = 1  # Maturità del forward in anni

In [53]:
# Calcolo di d1 nel modello di Bachelier
d1 = (X0 - K) / (sigma_X * np.sqrt(T))

# Calcolo del prezzo di un'opzione call nel modello di Bachelier
C = (X0 - K) * norm.cdf(d1) + sigma_X * np.sqrt(T) * norm.pdf(d1)

# Calcolo del CVA usando la formula chiusa
CVA = lgd * C * np.exp(-r * T)

CVA


102.63674629879772

In [54]:
# Numero di simulazioni Monte Carlo per il calcolo dell'intervallo di confidenza
num_simulations = 100000

# Simulazione dei percorsi del sottostante al tempo T usando il modello di Bachelier
XT_simulated = X0 + sigma_X * np.random.normal(0, np.sqrt(T), num_simulations)

# Calcolo dei payoff del forward per ciascun percorso
payoffs = np.maximum(XT_simulated - K, 0)

# Calcolo del valore attuale dei payoffs scontati al tasso r
pv_payoffs = payoffs * np.exp(-r * T)

# Calcolo del CVA come LGD * valore attuale medio del payoff
CVA_simulated = lgd * pv_payoffs

# Calcolo della media e della deviazione standard dei CVA simulati
CVA_mean = np.mean(CVA_simulated)
CVA_std = np.std(CVA_simulated)

# Calcolo dell'intervallo di confidenza al 98% per il CVA
confidence_level = 0.98
z_score = norm.ppf((1 + confidence_level) / 2)  # Z-score per il livello di confidenza del 98%
margin_of_error = z_score * CVA_std / np.sqrt(num_simulations)

# Intervallo di confidenza inferiore e superiore
ci_lower = CVA_mean - margin_of_error
ci_upper = CVA_mean + margin_of_error

(ci_lower, ci_upper, CVA_mean)

(102.09185998226981, 103.88260836515776, 102.98723417371379)